In [ ]:
import os
import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
def read_netcdf(path):
    data = nc.Dataset(path, 'r')
    return data

In [ ]:
# Recursive function for printing the netCDF file structure
def print_netcdf_structure(data, indent=0):
    indent_str = ' ' * indent
    
    # Print group name if it's a subgroup (indent > 0)
    if indent > 0:
        print(f"{indent_str}Group: {data.path}")
    
    # Print variables in the current group
    for var_name in data.variables:
        var = data.variables[var_name]
        print(f"{indent_str}Variable: {var_name}, shape={var.shape}, dtype={var.dtype}")
    
    # Recursively print subgroups
    for group_name in data.groups:
        print_netcdf_structure(data.groups[group_name], indent + 4)

In [ ]:
buoy2_data_path = 'data/measurements/2023-11-09_Buoy2_BSH_N-9.nc'
buoy2_data = read_netcdf(buoy2_data_path)

In [ ]:
# Print the structure of the netCDF file
print("NetCDF File Structure:")
print_netcdf_structure(buoy2_data)

## NetCDF File Structure

- **Variable**: `longitude`, shape=(1,), dtype=float32 ✅
- **Variable**: `latitude`, shape=(1,), dtype=float32 ✅
- **Variable**: `time`, shape=(67680,), dtype=float64 ✅
- **Variable**: `height`, shape=(6,), dtype=int32 ✅

### Group: /ZX_LIDAR_WLBZ_2 
- **Variable**: `wind_speed`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_from_direction`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_from_direction_availability`, shape=(67680, 1, 1, 6), dtype=float64
- **Variable**: `wind_speed_availability`, shape=(67680, 1, 1, 6), dtype=float64
- **Variable**: `wind_speed_min`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_speed_max`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_speed_stddev`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_count`, shape=(67680, 1, 1, 6), dtype=float64
- **Variable**: `turbulence_intensity`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `algorithm_bitmask`, shape=(67680, 1, 1, 6), dtype=float64

### Group: /METEO_WLBZ_2
#### Subgroup: /METEO_WLBZ_2/Vaisala Weather Station
- **Variable**: `rain_accumulation`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `rain_accumulation_calculated`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `rain_duration`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `rain_intensity`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `hail_accumulation`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `hail_duration`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `hail_intensity`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `precipitation_count`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `air_temperature`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `relative_humidity`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_pressure`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_density`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_density_moist`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `PTH_count`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `wind_speed_average`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_min`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_max`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_count`, shape=(67680, 1, 1), dtype=float64

#### Subgroup: /METEO_WLBZ_2/Airmar Weather Station
- **Variable**: `air_pressure`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_temperature`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_density`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_min`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_max`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_average`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_from_direction`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `count`, shape=(67680, 1, 1), dtype=float64

### Group: /ADD_DATA_WLBZ_2 ❌
#### Subgroup: /ADD_DATA_WLBZ_2/CTD ❌
- **Variable**: `sea_water_pressure`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_water_temperature`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_water_electrical_conductivity`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_water_salinity`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `speed_of_sound_in_sea_water`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_water_sigma_t`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `count`, shape=(67680, 1, 1), dtype=float64 ❌

#### Subgroup: /ADD_DATA_WLBZ_2/DEFI ❌
- **Variable**: `sea_water_temperature`, shape=(67680, 1, 1), dtype=float64 ❌

#### Subgroup: /ADD_DATA_WLBZ_2/Coda Octopus ❌
- **Variable**: `sea_surface_wave_significant_height`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_surface_wave_period_at_variance_spectral_density_maximum`, shape=(67680, 1, 1), dtype=float64 ❌

### Group: /ZX_LIDAR_WLBZ_6_MCP ✅
- **Variable**: `wind_from_direction`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_speed`, shape=(67680, 1, 1, 6), dtype=float64 ✅

### Group: /METEO_WLBZ_6_MCP ✅
#### Subgroup: /METEO_WLBZ_6_MCP/Airmar Weather Station ✅
- **Variable**: `air_pressure`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_temperature`, shape=(67680, 1, 1), dtype=float64 ✅

#### Subgroup: /METEO_WLBZ_6_MCP/Vaisala Weather Station ✅
- **Variable**: `air_pressure`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_temperature`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `relative_humidity`, shape=(67680, 1, 1), dtype=float64 ✅